In [1]:
!sudo apt update
!sudo apt-get install zstd
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,361 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,904 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu

In [2]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [3]:
!ollama pull gemma3:12b

In [ ]:
!ollama pull gpt-oss:20b

In [ ]:
!ollama pull nomic-embed-text

In [4]:
!ollama list

NAME          ID              SIZE      MODIFIED               
gemma3:12b    f4031aab637d    8.1 GB    Less than a second ago    


Ejecutar dos veces la siguiente celda (luego del restart se ejecuta nuevamente)

In [1]:
!pip3 install langchain langchain-community pypdf openai chromadb tiktoken sentence-transformers transformers ipykernel langchain-google-genai langchain_ollama langchain_experimental duckduckgo-search ddgs wikipedia pinecone langchain-pinecone

In [ ]:
# ----------------- CONFIGURACIÓN GOOGLE COLAB -------------------------------
import threading
import subprocess
import time
import os
import re
import sys

def start_ollama():
    # Iniciamos el proceso y redirigimos logs para evitar ensuciar la consola
    process = subprocess.Popen(
        ["ollama", "serve"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    # Esperamos a que el socket esté listo en lugar de un sleep fijo
    while True:
        try:
            import socket
            with socket.create_connection(("127.0.0.1", 11434), timeout=1):
                print("--- Ollama está listo ---")
                break
        except OSError:
            time.sleep(1)

thread = threading.Thread(target=start_ollama, daemon=True)
thread.start()
# ------------------------------------------------------------------

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.vectorstores import Chroma
from langchain_classic.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_community.tools import Tool
from google.colab import userdata
userdata.get('HUGGING_FACE_TOKEN')

DB_DIR = "/content/chroma_db_hp"
PDF_PATH = 'HP7.pdf'

if 'embeddings' not in locals() and 'embeddings' not in globals():
    print("--- Cargando HuggingFace Embeddings a la GPU por primera vez ---")
    embeddings = HuggingFaceEmbeddings(
        model_name="BAAI/bge-large-en-v1.5",
        model_kwargs={'device': 'cuda'}, # Forzamos explícitamente CUDA
        encode_kwargs={'normalize_embeddings': True}
    )
else:
    print("--- Embeddings ya cargados. Saltando para ahorrar VRAM ---")

if os.path.exists(DB_DIR):
    print("--- Cargando Base de Datos existente ---")
    vectorstore = Chroma(persist_directory=DB_DIR, embedding_function=embeddings)
else:
    print("--- Procesando PDF por primera vez ---")
    loader = PyPDFLoader(PDF_PATH)
    data = loader.load()

    def clean_text(text):
        # Limpieza más agresiva para eliminar encabezados/pies de página comunes
        text = re.sub(r'\n\s*\d+\s*\n', '\n', text)
        text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
        return " ".join(text.split())

    for doc in data:
        doc.page_content = clean_text(doc.page_content)

    # RECOMENDACIÓN: Chunk size menor para mayor precisión semántica
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200,
        chunk_overlap=500,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    chunks = text_splitter.split_documents(data)
    vectorstore = Chroma.from_documents(
        documents=chunks, embedding=embeddings, persist_directory=DB_DIR
    )
    print(f"--- DB creada con {len(chunks)} chunks ---")

# --- RETRIEVER CON FILTRO DE DIVERSIDAD ---
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 10, "fetch_k": 40, "lambda_mult": 0.5}
)

llm = ChatOllama(model="gemma3:12b", temperature=0.9, num_ctx=16384)

def search_hp_book(query: str) -> str:
    # HyDE mejorado con un sistema de respaldo
    hyde_prompt = f"Describe brevemente una escena de Harry Potter y las Reliquias de la Muerte sobre: {query}"
    try:
        hyde_response = llm.invoke(hyde_prompt).content
    except Exception:
        hyde_response = ""

    # Captura nombres propios pero evita palabras comunes al inicio de oración
    keywords = " ".join(re.findall(r'\b[A-Z][a-z]{2,}\b', query))

    combined_query = f"{query} {hyde_response} {keywords}".strip()
    docs = retriever.invoke(combined_query)

    if not docs: return "No se encontraron fragmentos relevantes."

    # Formateo con metadatos de página para mayor credibilidad
    results = []
    for d in docs:
        page = d.metadata.get('page', 'N/A')
        results.append(f"[Página {page}]: {d.page_content}")

    return "\n\n".join(results)

hp_tool = Tool(
    name="search_harry_potter",
    func=search_hp_book,
    description="Busca eventos y detalles en el libro 7. Si no encuentras algo, busca personajes presentes en la escena."
)

search_tool = DuckDuckGoSearchRun()
search_tool.name = "web_search"
search_tool.description = """
    Useful for searching current information on the internet, recent news,
    current events, or information that changes frequently.
    Input: question or query in natural language.
    Output: relevant search results.
    """

# --- TEMPLATE MEJORADO PARA EVITAR BUCLES ---
template = """Eres una IA experta en el séptimo libro de Harry Potter.

{tools}

REGLAS DE RAZONAMIENTO:
1. Si la búsqueda no arroja resultados, intenta buscar el lugar donde ocurrió el evento o personajes secundarios involucrados.
2. Si el usuario escribe mal un nombre (ej. 'Dobi'), corrígelo en tu búsqueda ('Dobby').
3. Siempre indica el número de página si está disponible.
4. No asumas hechos externos al libro 7.

Question: {input}
Thought: {agent_scratchpad}
"""
# Nota: LangChain React Agent requiere un formato muy específico.
prompt = PromptTemplate.from_template(
    "Answer the following questions as best you can. You have access to the following tools:\n\n"
    "{tools}\n\nUse the following format:\n\n"
    "Question: the input question you must answer\n"
    "Thought: you should always think about what to do\n"
    "Action: the action to take, should be one of [{tool_names}]\n"
    "Action Input: the input to the action\n"
    "Observation: the result of the action\n"
    "... (this Thought/Action/Action Input/Observation can repeat N times)\n"
    "Thought: I now know the final answer\n"
    "Final Answer: the final answer to the original input question (same lenguage), with explanation if it's required\n\n"
    "Begin!\n\nQuestion: {input}\nThought: {agent_scratchpad}"
)

agent = create_react_agent(llm, [hp_tool], prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=[hp_tool, search_tool],
    verbose=False,
    handle_parsing_errors=True,
    max_iterations=5 # Reducido para evitar loops infinitos costosos en Colab
)

def preguntar(pregunta: str, verbose: bool = False):
    try:
        response = agent_executor.invoke({"input": pregunta})
        print('-'*70)
        print(f"\nRespuesta Final:\n{response['output']}")
        print('-'*70)
    except Exception as e:
        print('-'*70)
        print(f"\nOcurrió un error: {e}")
        print('-'*70)


if __name__ == "__main__":
    while True:
        print('\n')
        print('-'*70)
        question = input("Haz tu pregunta: ")
        print('-'*70)
        preguntar(question)
        if input("¿Otra pregunta? (s/n): ").lower() == 'n':
            break

--- Ollama está listo ---
--- Embeddings ya cargados. Saltando para ahorrar VRAM ---
--- Procesando PDF por primera vez ---
--- DB creada con 1646 chunks ---


----------------------------------------------------------------------
Haz tu pregunta: Cómo se le conoce a la varita más poderosa de todas?
----------------------------------------------------------------------
----------------------------------------------------------------------

Respuesta Final:
La varita más poderosa de todas se conoce como la Varita de Saúco.
----------------------------------------------------------------------
¿Otra pregunta? (s/n): Cual es el nombre de la espada con la que matan a Nagini?


----------------------------------------------------------------------
Haz tu pregunta: Cual es el nombre de la espada con la que matan a Nagini?
----------------------------------------------------------------------
----------------------------------------------------------------------

Respuesta Final:
La espada ut

Pasar el verbose=True del AgentExecutor

In [ ]:
agent = create_react_agent(llm, [hp_tool], prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=[hp_tool, search_tool],
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5, # Reducido para evitar loops infinitos costosos en Colab
    early_stopping_method="generate"
)

In [5]:
preguntar('Cómo y en qué momento se besó Ron y Hermione?')



> Entering new AgentExecutor chain...
Action: search_harry_potter
Action Input: "Ron y Hermione beso"[Página 217]: . —Suena más espectacular de lo que ha sido en realidad —farfulló Ron. —Suele ocurrir así; hace años que intento explicártelo. Se acercaron al mismo tiempo y se abrazaron; Harry estrujó la espalda de la chaqueta de Ron, todavía empapada, y cuando se separaron dijo: —Y ahora tenemos que encontrar la tienda. Pero no les fue difícil. Pese a que la caminata por el oscuro bosque tras la cierva le había parecido muy larga, al llevar a Ron a su lado, el trayecto de regreso le resultó breve. Harry estaba deseando despertar a Hermione, y entró en la tienda con el corazón acelerado por la emoción, mientras que Ron se rezagó un poco. Comparado con la temperatura de la charca o el bosque, allí dentro hacía un calor delicioso; la única iluminación la proporcionaban las llamas azul turquesa, que seguían danzando en un cuenco que había en el suelo. Hermione dormía profundamente, acurru

In [6]:
preguntar('Qué personajes murieron en la batalla final?')



> Entering new AgentExecutor chain...
Action: search_harry_potter
Action Input: "muertes batalla final libro 7"[Página 392]: . Las imágenes de Fred, Lupin y Tonks muertos en el Gran Comedor volvieron a su mente, y se le cortó la respiración un instante; la muerte era impaciente… Pero Dumbledore lo había sobrestimado. Harry había fracasado, pues la serpiente sobrevivía. Por tanto, todavía quedaba un Horrocrux que ataría a Voldemort a la vida incluso después de que Harry Potter hubiera caído. Aunque lo cierto era que su sucesor tendría las cosas más fáciles. Harry se preguntó quién sería… Ron y Hermione sabrían lo que había que hacer, por supuesto… Seguramente por ese motivo Dumbledore había querido que él confiara en sus dos amigos, porque si se cumplía su verdadero destino demasiado pronto, ellos podrían continuar… Esos pensamientos golpeaban, como la lluvia contra los cristales de una ventana, sobre la dura superficie de la incontrovertible verdad: él debía morir. «Debo morir.» Tení

In [7]:
preguntar('Cómo matan a Harry?')



> Entering new AgentExecutor chain...
Thought: Necesito buscar en el libro 7 para encontrar la información sobre la muerte de Harry.
Action: search_harry_potter
Action Input: "muerte de Harry"[Página 371]: . —Lo lamento —dijo Voldemort con frialdad, y le dio la espalda. No sentía tristeza ni remordimiento. Había llegado la hora de abandonar aquella cabaña y hacerse cargo de la situación, provisto de una varita que ahora sí obedecería sus órdenes. Apuntó con ella a la estrellada jaula de la serpiente, que soltó a Snape y se deslizó hacia arriba, y el profesor quedó tendido en el suelo, con las heridas del cuello sangrando. Voldemort salió de la habitación sin mirar atrás, y la gran serpiente flotó tras él, encerrada en la enorme esfera. En el túnel, y de nuevo dueño de su mente, Harry abrió los ojos y se dio cuenta de que se había mordido tan fuerte los nudillos para no gritar que se había hecho sangre. Volvió a mirar por la estrecha rendija y logró ver un pie enfundado en una bota ne

In [8]:
preguntar('Cómo matan a Nagini?')



> Entering new AgentExecutor chain...
Action: search_harry_potter
Action Input: "muerte Nagini"[Página 414]: . Y entonces Neville también se movió. Con un rápido y fluido movimiento se libró de la maldición de inmovilidad total que lo aprisionaba, y el llameante sombrero se le cayó de la cabeza. Acto seguido sacó de su interior un objeto de plata con rubíes incrustados en la empuñadura… y de un solo tajo de espada degolló a la serpiente. La cabeza de Nagini salió despedida hacia arriba, girando sobre sí misma, reluciente a la luz que llegaba del vestíbulo. Voldemort abrió la boca para dar un grito de cólera que nadie pudo oír, y el cuerpo de la serpiente cayó a sus pies con un ruido sordo. Oculto bajo la capa, Harry hizo un encantamiento escudo entre Neville y Voldemort antes de que éste pudiera alzar la varita. Entonces, por encima de los gritos, los bramidos y las atronadoras pisadas de los batalladores gigantes, se oyó el grito de Hagrid: —¡¡Harry!! ¡¡Harry!! ¡¡¿Dónde está Harry?!

In [9]:
preguntar('Con qué objeto matan a Nagini?')



> Entering new AgentExecutor chain...
Action: search_harry_potter
Action Input: "Nagini muerte"[Página 365]: . Harry le lanzó un hechizo aturdidor a un mortífago enmascarado, pero no le acertó y estuvo a punto de darle a Neville, quien había aparecido de repente blandiendo una enorme Tentacula venenosa que se enrolló, gozosa, alrededor del primer mortífago que encontró y se dispuso a tirar de él. Harry, Ron y Hermione bajaron veloces por la escalinata de mármol. A su izquierda cayeron cristales, y el reloj de arena de Slytherin que registraba los puntos de la casa derramó sus esmeraldas por el suelo; al pisarlas, la gente resbalaba y perdía el equilibrio. Cuando los chicos llegaron al vestíbulo, dos cuerpos se precipitaron desde la barandilla de la escalinata, y una masa de color gris que parecía un animal trotó a cuatro patas hacia el vestíbulo y le hincó los dientes a uno de los que acababan de caer. —¡¡Nooo!! —chilló Hermione; su varita produjo un ensordecedor estallido y Fenrir G